In [44]:
totalValuesIntransctions= []
Transactions = []
c={}
l={}
discarded_transactions=[]


# Reading the txt file

In [31]:
def test():
    resultedFile= []
    totalRows = 0
    
    with open('../data.txt','r') as f: 
        lines = f.readlines()



    for line in lines: 
        if totalRows==0:
            totalRows=line
        else: 
            updatedFile = line.replace('\t','')
            updatedFile = updatedFile.replace(' ','')
            totalValuesIntransctions.append(updatedFile[1])
            updatedFile = updatedFile[2:]
            updatedFile=list(updatedFile.replace('\n',''))

            resultedFile.append(updatedFile)
        

    return resultedFile, totalRows
    


In [32]:
Transactions,totalRows=test()
print(Transactions)

[['1', '3', '4'], ['2', '3', '5'], ['1', '2', '3', '5'], ['2', '5']]


# Minimum Threshold 

In [42]:
minimum_confidence_percent = 50
minimumSupport = (int(totalRows)*minimum_confidence_percent)/100
print(minimumSupport)
print(totalValuesIntransctions)
print(Transactions)

2.0
['3', '3', '4', '2']
[['1', '3', '4'], ['2', '3', '5'], ['1', '2', '3', '5'], ['2', '5']]


# Counting the items

In [34]:
from collections import Counter
totalRowsCounter=0
totalItems = Counter()
for line in Transactions:

    if totalRowsCounter==0:
        totalItems=(Counter(line))
    else:
        totalItems=(Counter(line)) + totalItems
    totalRowsCounter = totalRowsCounter+1
    

totalItems=dict(totalItems)
print('C1')
print(totalItems)
print(list(totalItems.values()))


C1
{'2': 3, '5': 3, '1': 2, '3': 3, '4': 1}
[3, 3, 2, 3, 1]


In [43]:

initialSize = 1
c.update({initialSize : [[f] for f in totalItems.keys()] })
print(c)

{1: [['2'], ['5'], ['1'], ['3'], ['4']]}


In [ ]:
get_frequent(c[initialSize],Transactions,minimumSupport,discarded_transactions)

# Removing the item which is less than minimum threshold 

In [ ]:
itemsTobeRemoved =[]
for item in totalItems:
    if totalItems[item] <= minimumSupport:
        itemsTobeRemoved.append(item)
  
       
for key in itemsTobeRemoved:
    del totalItems[key]

print(totalItems)